In [ ]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium

print('Packages installed.')

In [2]:
pip install BeautifulSoup4

     |████████████████████████████████| 122kB 15.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
#import all libraries 
import numpy as np 
import pandas as pd 
import geocoder
import requests 
from bs4 import BeautifulSoup 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Libraries imported.


In [4]:
# Foursquare ID, Secret
CLIENT_ID = '0CYLAO2QKEZ3ACI42FV34YQNKQURFR3N5IPHJ2HROM2FFIQQ' 
CLIENT_SECRET = 'X4HTI4BSKIA1YLROGKFCS4PWWDZDDZBIYEGHOSYGLVUWRCVT'
VERSION = '20200518'
LIMIT = 100

In [5]:
#get the data
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

In [6]:
# collect data
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected')

Data Collected


In [7]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
# create datafreame
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [9]:
# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A\n,Not assigned\n,M2A\n,43.648690,-79.385440
1,M3A\n,North York\n,Parkwoods\n,43.752935,-79.335641
2,M4A\n,North York\n,Victoria Village\n,43.728102,-79.311890
3,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.650964,-79.353041
4,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n",43.723265,-79.451211
...,...,...,...,...,...
265,M9Y\n,Not assigned\n,M1Z\n,43.648690,-79.385440
266,M2Z\n,Not assigned\n,M3Z\n,43.648690,-79.385440
267,M4Z\n,Not assigned\n,M5Z\n,43.648690,-79.385440
268,M6Z\n,Not assigned\n,M7Z\n,43.648690,-79.385440


In [10]:
neighbors.shape

(270, 5)

In [11]:
neighborhood_name = neighbors.loc[0, 'Neighborhood'] # neighborhood name
neighborhood_latitude = neighbors.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighbors.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID, CLIENT_SECRET, VERSION, 43.7532586, -79.3296565, radius, LIMIT)

results = requests.get(url).json()
results

Latitude and longitude values of M2A
 are 43.648690000000045, -79.38543999999996.


{'meta': {'code': 200, 'requestId': '5ec38ea90cc1fd001b710f71'},
 'response': {'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.751976046055574,
          'lng': -79.33214044722958}],
        'distance': 245,
        'cc': 'CA'

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,TTC stop #8380,Bus Stop,43.752672,-79.326351
2,Variety Store,Food & Drink Shop,43.751974,-79.333114
3,Corrosion Service Company Limited,Construction & Landscaping,43.752432,-79.334661


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [22]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
toronto_venues = getNearbyVenues(names=neighborhoods,
                                 latitudes=neighbors['Latitude'],
                                 longitudes=neighbors['Longitude'])

M2A

Parkwoods

Victoria Village

Regent Park, Harbourfront

Lawrence Manor, Lawrence Heights

Queen's Park, Ontario Provincial Government

M9A

Malvern, Rouge

M3B

Parkview Hill, Woodbine Gardens

Garden District, Ryerson

Glencairn

M8B

West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale

Rouge Hill, Port Union, Highland Creek

M3C

Woodbine Heights

St. James Town

Humewood-Cedarvale

M8C

Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood

Guildwood, Morningside, West Hill

M3E

The Beaches

Berczy Park

Caledonia-Fairbanks

M8E

M1G

M3G

Leaside

Central Bay Street

Christie

M8G

M1H

Hillcrest Village

Bathurst Manor, Wilson Heights, Downsview North

Thorncliffe Park

Richmond, Adelaide, King

Dufferin, Dovercourt Village

M8H

M1J

Fairview, Henry Farm, Oriole

Northwood Park, York University

East Toronto

Harbourfront East, Union Station, Toronto Islands

Little Portugal, Trinity

M8J

M1K

Bayview Village

Downsview

The Danforth West, Riverd

In [24]:
# check the size of the resulting dataframe

print(toronto_venues.shape)
toronto_venues.head()

(540, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M2A\n,43.648690,-79.385440,Brookbanks Park,43.751976,-79.332140,Park
1,M2A\n,43.648690,-79.385440,TTC stop #8380,43.752672,-79.326351,Bus Stop
2,M2A\n,43.648690,-79.385440,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M2A\n,43.648690,-79.385440,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
4,Parkwoods\n,43.752935,-79.335641,Brookbanks Park,43.751976,-79.332140,Park


In [25]:
# check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt\n,4,4,4,4,4,4
"Alderwood, Long Branch\n",4,4,4,4,4,4
"Bathurst Manor, Wilson Heights, Downsview North\n",4,4,4,4,4,4
Bayview Village\n,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East\n",4,4,4,4,4,4
...,...,...,...,...,...,...
Willowdale\n,8,8,8,8,8,8
"Willowdale, Newtonbrook\n",4,4,4,4,4,4
Woodbine Heights\n,4,4,4,4,4,4


In [26]:
# find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 4 uniques categories.


In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Bus Stop,Construction & Landscaping,Food & Drink Shop,Park
0,M2A\n,0,0,0,1
1,M2A\n,1,0,0,0
2,M2A\n,0,0,1,0
3,M2A\n,0,1,0,0
4,Parkwoods\n,0,0,0,1


In [29]:
#df size
toronto_onehot.shape

(540, 5)

In [30]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Bus Stop,Construction & Landscaping,Food & Drink Shop,Park
0,Agincourt\n,0.25,0.25,0.25,0.25
1,"Alderwood, Long Branch\n",0.25,0.25,0.25,0.25
2,"Bathurst Manor, Wilson Heights, Downsview North\n",0.25,0.25,0.25,0.25
3,Bayview Village\n,0.25,0.25,0.25,0.25
4,"Bedford Park, Lawrence Manor East\n",0.25,0.25,0.25,0.25
...,...,...,...,...,...
126,Willowdale\n,0.25,0.25,0.25,0.25
127,"Willowdale, Newtonbrook\n",0.25,0.25,0.25,0.25
128,Woodbine Heights\n,0.25,0.25,0.25,0.25
129,York Mills West\n,0.25,0.25,0.25,0.25


In [31]:
# confirm the new size

toronto_grouped.shape

(131, 5)

In [32]:
# print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt
----
                        venue  freq
0                    Bus Stop  0.25
1  Construction & Landscaping  0.25
2           Food & Drink Shop  0.25
3                        Park  0.25


----Alderwood, Long Branch
----
                        venue  freq
0                    Bus Stop  0.25
1  Construction & Landscaping  0.25
2           Food & Drink Shop  0.25
3                        Park  0.25


----Bathurst Manor, Wilson Heights, Downsview North
----
                        venue  freq
0                    Bus Stop  0.25
1  Construction & Landscaping  0.25
2           Food & Drink Shop  0.25
3                        Park  0.25


----Bayview Village
----
                        venue  freq
0                    Bus Stop  0.25
1  Construction & Landscaping  0.25
2           Food & Drink Shop  0.25
3                        Park  0.25


----Bedford Park, Lawrence Manor East
----
                        venue  freq
0                    Bus Stop  0.25
1  Construction & Landsc

In [33]:
# sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Agincourt\n,Park,Food & Drink Shop,Construction & Landscaping
1,"Alderwood, Long Branch\n",Park,Food & Drink Shop,Construction & Landscaping
2,"Bathurst Manor, Wilson Heights, Downsview North\n",Park,Food & Drink Shop,Construction & Landscaping
3,Bayview Village\n,Park,Food & Drink Shop,Construction & Landscaping
4,"Bedford Park, Lawrence Manor East\n",Park,Food & Drink Shop,Construction & Landscaping


In [35]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [36]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M1A\n,Not assigned\n,M2A\n,43.648690,-79.385440,0,Park,Food & Drink Shop,Construction & Landscaping
1,M3A\n,North York\n,Parkwoods\n,43.752935,-79.335641,0,Park,Food & Drink Shop,Construction & Landscaping
2,M4A\n,North York\n,Victoria Village\n,43.728102,-79.311890,0,Park,Food & Drink Shop,Construction & Landscaping
3,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.650964,-79.353041,0,Park,Food & Drink Shop,Construction & Landscaping
4,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n",43.723265,-79.451211,0,Park,Food & Drink Shop,Construction & Landscaping


In [37]:
# visualize clusters

# create map
map_clusters = folium.Map(location=[43.67635739999999, -79.2930312], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:
# CLUSTER 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Not assigned\n,0,Park,Food & Drink Shop,Construction & Landscaping
1,North York\n,0,Park,Food & Drink Shop,Construction & Landscaping
2,North York\n,0,Park,Food & Drink Shop,Construction & Landscaping
3,Downtown Toronto\n,0,Park,Food & Drink Shop,Construction & Landscaping
4,North York\n,0,Park,Food & Drink Shop,Construction & Landscaping
...,...,...,...,...,...
130,Not assigned\n,0,Park,Food & Drink Shop,Construction & Landscaping
131,Not assigned\n,0,Park,Food & Drink Shop,Construction & Landscaping
132,Not assigned\n,0,Park,Food & Drink Shop,Construction & Landscaping
133,Not assigned\n,0,Park,Food & Drink Shop,Construction & Landscaping
